# **Calcul de similarité entre images**

Ce script est une copie arrangée du script du projet ModOAP disponible [ici](https://github.com/MODOAP/Similarite_image/blob/main/Modoap_Similarite_image.ipynb).

Ce notebook permet de calculer la similarité entre une image donnée et les images d'un corpus présent sur un Google Drive.
Il dispose de trois fonctionnalités :
1. Vectorisation d'un corpus / création d'un modèle à requêter
2. Requête des n plus proches images sur un modèle importé, à partir d'une image donnée en entrée et repérage des doublons
3. Sauvegarde des images similaires, distances et des doublons dans un fichier Excel de sortie

Il implémente la bibliothèque turicreate : https://github.com/apple/turicreate

**Le mode GPU est nécessaire (Runtime -> Change Runtime Type -> GPU)**

In [ ]:
#@markdown # Connexion à un compte Google Drive et installation des pré-requis

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

#@markdown Résultat : création d'un dossier Outils_Modoap sur le Drive qui servira à stocker les modèles générés.

from google.colab import drive
import os

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Le Drive est déjà monté")

if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Similarite"):
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Similarite/Doublons')
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Similarite/Modeles')

!apt install libnvrtc8.0
!pip uninstall -y mxnet
!pip install mxnet-cu100
!pip install turicreate

import turicreate as tc
from IPython.display import display
from IPython.display import HTML
from PIL import Image
from io import BytesIO
import glob
import random
import base64
import pandas as pd
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
from google.colab import drive
from time import gmtime, strftime
import shutil

def get_thumbnail(path):
  i = Image.open(path)
  i.thumbnail((150, 150), Image.LANCZOS)
  return i
  
def get_thumbnail_from_image(img):
  i = img.copy()
  i.thumbnail((150, 150), Image.LANCZOS)
  return i

def image_base64(im):
  if isinstance(im, str):
    im = get_thumbnail(im)
  with BytesIO() as buffer:
    im.save(buffer, 'jpeg')
    return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
  return f'<img style="display:inline;margin:1px" src="data:image/jpeg;base64,{image_base64(im)}">'
  
def preview_images(reference_data, num_previews=30):
  images = list(map(lambda x:image_formatter(get_thumbnail(x)), reference_data[0:num_previews]['path']))
  display(HTML(''.join(images)))


# Use all GPUs (default)
#tc.config.set_num_gpus(-1)

# Use only 1 GPU
tc.config.set_num_gpus(1)

# Use CPU
#tc.config.set_num_gpus(0)

In [ ]:
#@markdown # Identification du corpus d'images

#@markdown Le corpus doit être sous forme d'un dossier sur le Google Drive contenant directement les images.

#@markdown Indiquer le chemin vers le dossier d'images sur le Google Drive :
chemin_vers_dossier_dimages = '/content/drive/My Drive/Images/'#@param {type:"string"}

#@markdown En guise de résultat à cette cellule s'affiche le nombre de fichiers jpg présents dans le dossier Google Drive.

corpus = chemin_vers_dossier_dimages

from os import listdir
from os.path import isfile, join

liste_fichiers = [f for f in listdir(chemin_vers_dossier_dimages) if isfile(join(chemin_vers_dossier_dimages, f))]
print(len(liste_fichiers))

df = pd.DataFrame(liste_fichiers, columns = ['Image traitée'])
df = df.sort_values(by=['Image traitée'])

In [ ]:
#@markdown # Création et sauvegarde du modèle à partir du corpus

#@markdown Le modèle créé est enregistré sur Google Drive dans /content/drive/My Drive/Outils_Modoap/Similarite/Modeles/ .

# Load images from a folder
reference_data = tc.image_analysis.load_images(corpus)
reference_data = reference_data.add_row_number()
# From the path-name, create a label column
reference_data['label'] = reference_data['path'].apply(lambda path: path.split('/')[-2])
reference_data.groupby('label', [tc.aggregate.COUNT]).sort("Count", ascending = False)
# Save the SFrame for future use
reference_data.save('/content/drive/My Drive/Outils_Modoap/Similarite/Modeles/reference_data.sframe')
reference_data.groupby('label', [tc.aggregate.COUNT]).sort("Count", ascending = False)
reference_data.head()

# create an image similarity model using the data
model = tc.image_similarity.create(reference_data)
# saving the model
%cd /content/drive/My Drive/Outils_Modoap/Similarite/Modeles
nom_modele = "ModeleSimilarite_"+str(strftime("%Y-%m-%d %H:%M:%S", gmtime())).replace(" ","_")+".zip"
model.save('./image_similarity.model')
!zip -r ./$nom_modele ./image_similarity.model ./reference_data.sframe
!rm -r ./image_similarity.model
!rm -r ./reference_data.sframe

In [ ]:
#@markdown # Import d'un modèle depuis Google Drive (optionnel)

#@markdown Si le modèle vient d'être créé et sauvegardé à l'étape précédente, il n'est pas nécessaire de l'importer. Si le notebook a été déconnecté entre-temps, exécuter cette cellule.

#@markdown Indiquer le chemin absolu vers le dossier zip contenant le modèle enregistré sur Google Drive : 
chemin_vers_modele = '/content/drive/My Drive/Outils_Modoap/Similarite/Modeles/monmodele.zip/'#@param {type:"string"}

fichier = chemin_vers_modele
%cd /content/drive/My Drive/Outils_Modoap/Similarite/Modeles
ds = ZipFile(fichier)
ds.extractall()
print("Modèle téléchargé et extrait")

reference_data = tc.load_sframe('/content/drive/My Drive/Outils_Modoap/Similarite/Modeles/reference_data.sframe')
model = tc.load_model('/content/drive/My Drive/Outils_Modoap/Similarite/Modeles/image_similarity.model')
print("Modèle et Sframe importés")

In [ ]:
#@markdown # Lancer la requête d'images similaires sur le modèle créé

#@markdown Indiquer le nombre d'images similaires à rapporter :
nb_images_similaires = 10 #@param {type:"integer"}

#@markdown Indiquer le chemin vers le fichier de sortie sur le Google Drive :
chemin_vers_fichier_sortie = '/content/drive/My Drive/resultat.xlsx'#@param {type:"string"}

userk = nb_images_similaires

for nom in liste_fichiers:
  image_url = chemin_vers_dossier_dimages + nom
  print(image_url)
  sample_image = tc.Image(image_url)
  image = image_url.replace(chemin_vers_dossier_dimages, "")
  matched_row = df.index[df['Image traitée'] == image]
  si_height = sample_image.height 
  si_width = sample_image.width
  if int(si_height) > 600 :
    sample_image = tc.image_analysis.resize(sample_image, 320, 240)
    #display(sample_image)
  else : 
    #display(sample_image)
    print("--------------------------------------------------------------")

  query_results = model.query(sample_image, k=int(userk))
  similar_rows = query_results[query_results['query_label'] == 0]['reference_label']
  doublons_potentiels = [idi for idi in similar_rows if query_results[query_results['reference_label'] == idi]['distance'][0] < 10]
  similar_rows = [i for i in similar_rows if i not in doublons_potentiels]
  doublons_data = reference_data.filter_by(doublons_potentiels, 'id')
  similar_rows_data = reference_data.filter_by(similar_rows, 'id')
  similar_rows_data = similar_rows_data.filter_by(doublons_potentiels, 'id', exclude = True)

  # Struction des doublons récupérés avant ajout dans le dataframe créé auparavant
  if len(doublons_data) >= 1 :
    liste_doublons = []
    for doublon in doublons_potentiels :
      chemin = doublons_data[doublons_data['id'] == doublon]['path'][0]
      liste_doublons.append(chemin)

  for i in liste_doublons:
    if i == image_url:
      liste_doublons.remove(i)

  doublons_df = ' ; '.join([str(elem) for elem in liste_doublons ])
  doublons_df = doublons_df.replace(chemin_vers_dossier_dimages, "")
  df.at[matched_row, 'Doublon(s)'] = doublons_df

  try:
    if os.path.isfile(similar_rows_data[0]["path"]):
      liste_images_similaires = []
      liste_distances = []
      for idi in similar_rows :
        chem1 = similar_rows_data[similar_rows_data['id'] == idi]['path'][0]
        way = chem1.replace(chemin_vers_dossier_dimages, "")
        dist = query_results[query_results['reference_label'] == idi]['distance'][0]
        liste_images_similaires.append(way)
        liste_distances.append(dist)
    else:
      for row in similar_rows_data :
        print(row["path"])
      print("Le corpus n'a pas été importé, impossible d'afficher les images")

    distances_df = ' ; '.join([str(elem) for elem in liste_distances ])
    images_similaires_df = ' ; '.join([str(elem) for elem in liste_images_similaires ])
    df.at[matched_row, 'Images similaires'] = images_similaires_df
    df.at[matched_row, 'Distance'] = distances_df

  except IndexError:
    print("Aucune image similaire trouvée")

df.to_excel(chemin_vers_fichier_sortie)